# SQLAlchemy 1.4 / 2.0 Tutorial

## Establishing Connectivity - the Engine

In [14]:
import os.path
import yaml
from sqlalchemy import create_engine, text

config_path = os.path.expanduser('~\\.sqlalchemy\\config.yaml')

with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

db_path = (
    f"postgresql://{config['username']}:{config['password']}"
    f"@{config['host']}:{config['port']}/{config['db_name']}"
)
engine = create_engine(db_path, echo=True, future=True)

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())


2021-11-18 13:58:48,954 INFO sqlalchemy.engine.Engine select version()
2021-11-18 13:58:48,954 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-18 13:58:48,955 INFO sqlalchemy.engine.Engine select current_schema()
2021-11-18 13:58:48,956 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-18 13:58:48,957 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-11-18 13:58:48,958 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-18 13:58:48,959 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 13:58:48,960 INFO sqlalchemy.engine.Engine select 'hello world'
2021-11-18 13:58:48,961 INFO sqlalchemy.engine.Engine [generated in 0.00126s] {}
[('hello world',)]
2021-11-18 13:58:48,963 INFO sqlalchemy.engine.Engine ROLLBACK


## Working with Transactions and the DBAPI

### Committing Changes

In [15]:
# commit as you go
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE FUNC( x integer, y integer )"))
    conn.execute(text("INSERT INTO FUNC (x,y) VALUES (:x, :y)"),
                 [{"x": 1, "y": 1}, {"x": 2, "y": 4}])
    conn.commit()

# begin once
with engine.begin() as conn:
    conn.execute(text("INSERT INTO FUNC (x,y) VALUES (:x, :y)"),
                 [{"x": 6, "y": 8}, {"x": 9, "y": 10}])


2021-11-18 15:15:22,626 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 15:15:22,627 INFO sqlalchemy.engine.Engine CREATE TABLE FUNC( x integer, y integer )
2021-11-18 15:15:22,627 INFO sqlalchemy.engine.Engine [generated in 0.00121s] {}
2021-11-18 15:15:22,652 INFO sqlalchemy.engine.Engine INSERT INTO FUNC (x,y) VALUES (%(x)s, %(y)s)
2021-11-18 15:15:22,653 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ({'x': 1, 'y': 1}, {'x': 2, 'y': 4})
2021-11-18 15:15:22,655 INFO sqlalchemy.engine.Engine COMMIT
2021-11-18 15:15:22,658 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 15:15:22,659 INFO sqlalchemy.engine.Engine INSERT INTO FUNC (x,y) VALUES (%(x)s, %(y)s)
2021-11-18 15:15:22,659 INFO sqlalchemy.engine.Engine [cached since 0.007468s ago] ({'x': 6, 'y': 8}, {'x': 9, 'y': 10})
2021-11-18 15:15:22,663 INFO sqlalchemy.engine.Engine COMMIT


### Basics of Statement Execution

In [17]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x,y FROM FUNC;"))
    for row in result:
        print(f"x: {row.x} y: {row.y}")

    result = conn.execute(text("SELECT x,y FROM FUNC;"))
    for dict_row in result.mappings():
        x = dict_row['x']
        y = dict_row['y']
        print(x, y)

    result = conn.execute(text("SELECT x,y FROM FUNC where y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x} y: {row.y}")

    conn.execute(text("INSERT INTO FUNC(x,y) VALUES (:x, :y)"),
                 [{"x": 11, "y": 12}, {"x": 13, "y": 14}])
    conn.commit()

    # bundling parameters
    stmt = text(
        "SELECT x, y FROM FUNC WHERE y > :y ORDER BY x,y"
        ).bindparams(y=6)
    result = conn.execute(stmt)
    for row in result:
        print(f"x: {row.x} y: {row.y}")


2021-11-18 15:56:35,409 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 15:56:35,410 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC;
2021-11-18 15:56:35,410 INFO sqlalchemy.engine.Engine [generated in 0.00166s] {}
x: 1 y: 1
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2021-11-18 15:56:35,412 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC;
2021-11-18 15:56:35,413 INFO sqlalchemy.engine.Engine [cached since 0.004293s ago] {}
1 1
2 4
6 8
9 10
2021-11-18 15:56:35,415 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC where y > %(y)s
2021-11-18 15:56:35,415 INFO sqlalchemy.engine.Engine [generated in 0.00051s] {'y': 2}
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2021-11-18 15:56:35,420 INFO sqlalchemy.engine.Engine INSERT INTO FUNC(x,y) VALUES (%(x)s, %(y)s)
2021-11-18 15:56:35,421 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ({'x': 11, 'y': 12}, {'x': 13, 'y': 14})
2021-11-18 15:56:35,422 INFO sqlalchemy.engine.Engine COMMIT
2021-11-18 15:56:35,423 INFO sqlalchemy.engine.Engine BEGIN (impl

### Executing with an ORM Session

In [18]:
from sqlalchemy.orm import Session
stmt = text(
    "SELECT x, y FROM FUNC WHERE y > :y ORDER BY x, y"
).bindparams(y=6)

with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
        print(f"x: {row.x} y: {row.y}")

    result = session.execute(
        text("UPDATE FUNC SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}]
    )
    session.commit()  # To save, you should commit


2021-11-18 16:02:08,297 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine SELECT x, y FROM FUNC WHERE y > %(y)s ORDER BY x, y
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine [generated in 0.00039s] {'y': 6}
x: 6 y: 8
x: 9 y: 10
x: 11 y: 12
x: 13 y: 14
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine UPDATE FUNC SET y=%(y)s WHERE x=%(x)s
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine [generated in 0.00036s] ({'y': 11, 'x': 9}, {'y': 15, 'x': 13})
2021-11-18 16:02:08,319 INFO sqlalchemy.engine.Engine COMMIT
